# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# %%capture
!pip install kmapper matplotlib numpy scikit_learn umap umap-learn

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.5 MB/s eta 0:00:00
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3542 sha256=20ebc5afe2313ab4c98be0b23d32fc0a0b577726c44825879c4e39c068982636
  Stored in directory: /root/.cache/pip/wheels/15/f1/28/53dcf7a309118ed35d810a5f9cb995217800f3f269ab5771cb
Successfully built umap


In [3]:
import pickle

import kmapper as km
from kmapper.jupyter import display
import umap
import sklearn
import sklearn.manifold as manifold
import numpy as np
import matplotlib.pyplot as plt

# load sae features

In [4]:
fn = 'ts-1L-21M_Wdec'
# fn = 'ts-2L-33M_Wdec'
file_path = f'/content/drive/MyDrive/{fn}.pkl'
with open(file_path, 'rb') as f:
    feature_weights = pickle.load(f)

In [5]:
data = feature_weights.detach().cpu().numpy()
data.shape

(16384, 1024)

# load labels

In [9]:
import json
with open('feature_top_samps_lst_1L_16k.json', 'rb') as f:
# with open('feature_top_samps_lst_2L_MLP0.json', 'rb') as f:
    feat_snip_dict = json.load(f)

In [10]:
import re

def extract_tagged_word(s):
    # Define the regex pattern to match the tagged word
    pattern = r'\[bold u dark_orange\](.*?)\[/\]'

    # Search for the pattern in the string
    match = re.search(pattern, s)

    # If a match is found, return the captured group (the word inside the tags)
    if match:
        return match.group(1)
    else:
        return None

In [11]:
fList_model_A = []
for feat_dict in feat_snip_dict:
    text = feat_dict['strings'][0]
    result = extract_tagged_word(text)
    fList_model_A.append(result)
    # out_str = ''
    # for text in feat_dict['strings']:
    #     result = extract_tagged_word(text)
    #     out_str += result + ', '
    # fList_model_A.append(out_str)

In [12]:
fList_model_A[:5]

[' both', ' with', ' by', ' you', ' ashamed']

In [13]:
fList_model_A = np.array(fList_model_A)
len(fList_model_A)

16384

# Mapper

In [14]:
from sklearn.decomposition import PCA

In [20]:
mapper = km.KeplerMapper(verbose=1) # initialize mapper

# project data into 2D subspace via 2 step transformation, 1)isomap 2)UMAP
# projected_data = mapper.fit_transform(data, projection=[manifold.Isomap(n_components=100, n_jobs=-1), umap.UMAP(n_components=2,random_state=1)])
# projected_data = mapper.fit_transform(data, projection=PCA(n_components=2))
projected_data = mapper.fit_transform(data, projection="l2norm")

# cluster data using DBSCAN
graph = mapper.map(projected_data, data, clusterer=sklearn.cluster.DBSCAN(metric="cosine"))

KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: l2norm
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (16384, 1024)

..Projecting data using: l2norm

..Scaling with: MinMaxScaler()

Mapping on data shaped (16384, 1024) using lens shaped (16384, 1)

Creating 10 hypercubes.

Created 42 edges and 59 nodes in 0:00:01.662261.


In [21]:
# define an excessively long filename (helpful if saving multiple Mapper variants for single dataset)
fileID = fn + '_projection=' + graph['meta_data']['projection'].split('(')[0] + '_' + \
'n_cubes=' + str(graph['meta_data']['n_cubes']) + '_' + \
'perc_overlap=' + str(graph['meta_data']['perc_overlap']) + '_' + \
'clusterer=' + graph['meta_data']['clusterer'].split('(')[0] + '_' + \
'scaler=' + graph['meta_data']['scaler'].split('(')[0]

fileID

'ts-1L-21M_Wdec_projection=l2norm_n_cubes=10_perc_overlap=0.1_clusterer=DBSCAN_scaler=MinMaxScaler'

In [22]:
labels = list(range(data.shape[0]))
labels = np.array(labels)

In [23]:
mapper.visualize(graph,
                path_html=fileID + ".html",
                title=fileID,
                custom_tooltips =  fList_model_A,
                # custom_tooltips = labels,
                # color_values = np.log(per_return+1),
                color_function_name = 'test',
                node_color_function = np.array(['average', 'std', 'sum', 'max', 'min']))

Wrote visualization to: ts-1L-21M_Wdec_projection=l2norm_n_cubes=10_perc_overlap=0.1_clusterer=DBSCAN_scaler=MinMaxScaler.html


/usr/local/lib/python3.10/dist-packages/kmapper/kmapper.py:872: UserWarning: `color_function_name` was set -- however, no `color_values` were passed, so default color_values were computed based on row order, and the passed `color_function_name` will be set as their label. This may be unexpected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/kmapper/visuals.py:344: RuntimeWarning: invalid value encountered in scalar divide
  height = np.floor(((bar / max_bucket_value) * 100) + 0.5)
/usr/local/lib/python3.10/dist-packages/kmapper/visuals.py:345: RuntimeWarning: invalid value encountered in scalar divide
  perc = round((bar / sum_bucket_value) * 100.0, 1)


'<!DOCTYPE html>\n<html>\n\n<head>\n  <meta charset="utf-8">\n  <meta name="generator" content="KeplerMapper">\n  <title>ts-1L-21M_Wdec_projection=l2norm_n_cubes=10_perc_overlap=0.1_clusterer=DBSCAN_scaler=MinMaxScaler | KeplerMapper</title>\n\n  <link rel="icon" type="image/png" href="http://i.imgur.com/axOG6GJ.jpg" />\n\n  <link href=\'https://fonts.googleapis.com/css?family=Roboto+Mono:700,300\' rel=\'stylesheet\' type=\'text/css\'>\n  <style>* {\n  margin: 0;\n  padding: 0;\n}\n\nhtml, body {\n  height: 100%;\n}\n\nbody {\n  font-family: "Roboto Mono", "Helvetica", sans-serif;\n  font-size: 14px;\n}\n\n#logo {\n  width:  85px;\n  height: 85px;\n}\n\n#display {\n  color: #95A5A6;\n  background: #212121;\n}\n\n#header {\n  background: #111111;\n}\n\n#print {\n  color: #000;\n  background: #FFF;\n}\n\nh1 {\n  font-size: 21px;\n  font-weight: 300;\n  font-weight: 300;\n}\n\nh2 {\n  font-size: 18px;\n  padding-bottom: 20px;\n  font-weight: 300;\n}\n\nh3 {\n  font-size: 14px;\n  font-wei

In [24]:
from google.colab import files
files.download(fileID + ".html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>